In [ ]:
pip install transformers datasets peft accelerate bitsandbytes torch sentencepiece

4

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "./your_model_folder"  # Path to your downloaded model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token

In [2]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                  # Rank of LoRA matrices (lower = less memory)
    lora_alpha=32,        # Scaling factor (higher = stronger adaptation)
    target_modules=["c_attn"],  # Layers to apply LoRA
    lora_dropout=0.05,    # Dropout for regularization
    bias="none",          # No bias tuning
    task_type="CAUSAL_LM" # For causal language models
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Check trainable params (~0.1% of total)

ModuleNotFoundError: No module named 'peft'

In [4]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("software_qa.csv")  # Columns: ["question", "answer"]
dataset = Dataset.from_pandas(df)

ModuleNotFoundError: No module named 'datasets'

In [3]:
def tokenize_function(examples):
    return tokenizer(examples["content"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

NameError: name 'dataset' is not defined

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora_finetuned",
    per_device_train_batch_size=4,  # Reduce if OOM errors occur
    gradient_accumulation_steps=4,   # Simulate larger batch size
    num_train_epochs=3,
    learning_rate=3e-4,             # Higher than full fine-tuning
    logging_steps=10,
    save_steps=500,
    fp16=True,                      # Use mixed precision (GPU)
    optim="adamw_torch",            # Optimizer
    report_to="none",               # Disable WandB/MLflow logging (optional)
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
model.save_pretrained("./lora_adapters")

In [ ]:
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(model_path)
# Merge with LoRA
merged_model = PeftModel.from_pretrained(base_model, "./lora_adapters").merge_and_unload()
merged_model.save_pretrained("./merged_model")

In [ ]:
from peft import PeftModel

# Load base model + LoRA
model = AutoModelForCausalLM.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, "./lora_adapters")

# Generate responses
input_text = "How do I reverse a linked list in Python?"
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load base + LoRA-adapted model
base_model = "microsoft/phi-2"
adapter_path = "./phi2-finetuned/checkpoint-3"

tokenizer = AutoTokenizer.from_pretrained(base_model)
base = AutoModelForCausalLM.from_pretrained(base_model, device_map="auto")
model = PeftModel.from_pretrained(base, adapter_path)



In [ ]:
# Chat function
def chat(prompt, max_length=300):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


# Try a prompt
prompt = "can you explain, what is machine learning?"
response = chat(prompt)
print(response)